In [1]:
import time
import tweepy
import pandas as pd
from IPython.display import clear_output
import random
import pickle

In [16]:
# create connection with Twitter API

CONSUMER_KEY = 'J3uhwWHdmSt3uD69ry8r2kc3B'
CONSUMER_SECRET = 'LeuM7dVfFbXH0bc5fFkZIxlNiUXyjIx4Kcjh58HeWs2TrLijDx'
ACCESS_TOKEN = '327497511-5dNjiGaTQHASljoxNr1qlMOlHrrSB21HBFBDjx2E'
ACCESS_TOKEN_SECRET = 'tHLqwlhHYJWptW3femyq0rbMW6ZItu5yLQ3DFGJqJ8Xlg'

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth)

In [17]:
ids = pd.read_csv('data/legitimate_users.txt', sep = '\t')['614'].tolist()

In [18]:
tweets = []
cont, errors = 0, 0

for user in ids:
    try:
        stuff = api.user_timeline(user_id = user, count = 100, include_rts = True, tweet_mode="extended")
        #tweet.is_quote_status indica se nel tweet c'è il riferimento ad un altro tweet

        for tweet in stuff:
            tweet._json['user_id'] = tweet._json['user']['id']
            if len(tweet._json['entities']['urls']) != 0:
                tweet._json['url'] = tweet._json['entities']['urls'][0]['expanded_url']
            else:
                 tweet._json['url'] = None
            del tweet._json['user'], tweet._json['entities']

            tweets.append(tweet._json)
    except:
        errors += 1
    cont += 1
    clear_output()
    print(str(cont) + ' / ' + str(len(ids)) + '   ---   errors: ' + str(errors))
    
    if cont % 500 == 0:
        path = 'humans_tweets' + str(cont-500) + '-' + str(cont) + '.tw'
        pickle.dump( tweets, open( path , "wb" ) )
        tweets = []
        print('sleeping...')
        time.sleep(900)

19275 / 19275   ---   errors: 5889


In [19]:
path = 'humans_tweets' + str(cont-500) + '-' + str(cont) + '.tw'
pickle.dump( tweets, open( path , "wb" ) )

In [2]:
path = 'humans_tweets18775-19275.tw'
tweets = pickle.load(open(path,"rb"))

In [3]:
for i in range(1,39):
    path = 'humans_tweets' + str(i*500-500) + '-' + str(i*500) + '.tw'
    tweets = tweets + pickle.load(open(path,"rb"))
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38


In [4]:
df = pd.DataFrame.from_dict(tweets)

In [5]:
# rimuove tutte le interruzioni (spero)

df['full_text'] = df['full_text'].apply(lambda x: x.replace('\r', ' '))
df['full_text'] = df['full_text'].apply(lambda x: x.replace('\n', ' '))
df['full_text'] = df['full_text'].apply(lambda x: x.replace('\t', ' '))

In [ ]:
df.to_csv('data/bon/humans/tweets.csv', index=False, sep='\t', encoding='utf-8-sig')

In [ ]:
len(df)